# Week 1 recap

In [1]:
import json
from pathlib import Path
from typing import List

import altair as alt
from espn_api.football import League
import pandas as pd

In [2]:
FNAME = './data/week_01_points.json'
DATA_DIR = Path.cwd() / 'data'

TEAMS = {
    1: 'Fully  Maccinated',
    2: "Rippin' Swigs",
    3: 'Seattle Fantasy Team',
    10: 'Spenny Willy',
    11: 'Raiders Are Good',
    12: 'Herb Your Enthusiasm',
    13: 'Too Many Cooks',
    15: 'Elite starts  with Eli',
    16: 'Draftin Herbert First Pick',
    17: 'Scooty Lewis and the News',
    18: 'Team Daktus Jack',
    19: 'Federal Way Yu',
}

LEAGUE_ID = 17588244
YEAR = 2021

In [3]:
def load_or_query_draft_amounts() -> dict:
    """Loads draft data from ESPN and saves or loads from file"""
    fout = DATA_DIR / 'draft_bid_amounts.json'

    if fout.exists():
        print('loading file on disk')
        with open(fout) as f:
            draft_amounts = json.load(f)
    else:
        print('pulling data from ESPN')
        league = League(league_id=LEAGUE_ID, year=YEAR)
        draft = league.draft
        draft_amounts = {str(p.playerId): p.bid_amount for p in draft}
        assert len(draft_amounts) == 180

        with open(fout, 'wt') as f:
            json.dump(draft_amounts, f)

    return draft_amounts

DRAFT_AMOUNTS = load_or_query_draft_amounts()

loading file on disk


In [4]:
def flatten_json(teams: List[dict]) -> pd.DataFrame:
    flattened = []
    for team_id, team in teams.items():
        for pos, players in team.items():
            for p in players:
                row = []
                row = [team_id] + list(p.values())
                flattened.append(row)

    df = pd.DataFrame(flattened, columns=['fantasy_team_id', 'player_id', 'name', 'team', 'pos',
                                          'slot_position', 'injury_status', 'actual_points',
                                          'projected_points', 'points_breakdown'])
    # drop for now
    df = df.drop('points_breakdown', axis=1)
    df['points_diff'] = df['actual_points'] - df['projected_points']
    df['fantasy_team_name'] = df['fantasy_team_id'].apply(lambda s: TEAMS.get(int(s)))
    df['draft_price'] = df['player_id'].apply(lambda s: DRAFT_AMOUNTS.get(str(s), 0))

    reordered = ['fantasy_team_id', 'fantasy_team_name', 'player_id', 'name', 'team',
                 'pos', 'slot_position', 'injury_status', 'actual_points',
                 'projected_points', 'points_diff', 'draft_price']

    return df[reordered]

In [5]:
with open(FNAME) as f:
    teams = json.load(f)

df = flatten_json(teams)

df.head()

,fantasy_team_id,fantasy_team_name,player_id,name,team,pos,slot_position,injury_status,actual_points,projected_points,points_diff,draft_price
0,16,Draftin Herbert First Pick,3043078,Derrick Henry,TEN,RB,RB,ACTIVE,9.2,16.53,-7.33,68
1,16,Draftin Herbert First Pick,4259545,D'Andre Swift,DET,RB,RB,ACTIVE,20.4,11.63,8.77,22
2,16,Draftin Herbert First Pick,4035538,David Montgomery,CHI,RB,RB/WR/TE,ACTIVE,18.3,13.47,4.83,27
3,16,Draftin Herbert First Pick,16733,Odell Beckham Jr.,CLE,WR,WR,OUT,0.0,0.00,0.00,15
4,16,Draftin Herbert First Pick,4239993,Tee Higgins,CIN,WR,WR,ACTIVE,13.8,10.26,3.54,14


## Projects and actual points

Binned by original draft price

In [6]:
m = (df['projected_points'] > 0) #& (df['slot_position'] == 'WR')

tooltip_cols = ['name', 'team', 'slot_position', 'projected_points', 'actual_points',
                'points_diff', 'fantasy_team_name', 'draft_price']

chart = (alt.Chart(df[m])
 .mark_point()
 .encode(x='projected_points:Q', y='actual_points:Q',
         color='pos',
         size='draft_price',
         tooltip=tooltip_cols)
 .properties(width=500, height=500))

chart.interactive() + chart.transform_regression('projected_points', 'actual_points').mark_line()

alt.LayerChart(...)

## winners and losers

Top 20 best and top 20 worst point diffs (actual vs projected)

In [7]:
# players with projected points
m = (df['projected_points'] > 0) #& (df['slot_position'] != 'BE')

top = df[m].sort_values('points_diff', ascending=False).head(20)
bottom = df[m].sort_values('points_diff', ascending=True).head(20)

winners_losers = pd.concat([top, bottom])

In [8]:
tooltip_cols = ['name', 'team', 'slot_position', 'projected_points', 'actual_points',
                'points_diff', 'fantasy_team_name', 'draft_price']

(alt.Chart(winners_losers)
 .mark_bar()
 .encode(x='points_diff', y='name', color='slot_position', tooltip=tooltip_cols)
 .properties(width=400, height=600))

alt.Chart(...)